In [2]:
import pandas as pd 

import json

import numpy as np


from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [3]:
billboard_data = "billboard_original.csv"

In [4]:
billboard_df = pd.read_csv(billboard_data)
billboard_df

,name,artists,weekly_rank,peak_position,weeks_on_chart,week,date,genre,writing_credits,lyrics,features,grammy_win
0,Old Town Road,"Lil Nas,",1,1.0,7.0,7/6/19,5-Apr-19,"Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus,NO
1,Senorita,"Shawn Mendes, Camila Cabello",2,NaN,NaN,7/6/19,21-Jun-19,Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN,NO
2,Bad Guy,Billie Eilish,3,2.0,13.0,7/6/19,29-Mar-19,"Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN,NO
3,Talk,Khalid,4,3.0,20.0,7/6/19,7-Feb-19,"Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN,NO
4,I Don't Care,"Ed Sheeran, Justin Bieber",5,2.0,7.0,7/6/19,10-May-19,"Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
97220,Smile,Vitamin C,95,NaN,NaN,7/12/99,NaN,"Jamaica,Pop","Colleen fitzpatrick, Josh deutsch","Smile \nHahaha\nAlright, yeah\nAlright\nFirst ...",Lady Saw,NO
97221,Heavy,Collective Soul,96,73.0,20.0,7/12/99,NaN,"Hockey,Gaming,Soundtrack,Rock",Collective soul,Heavy \nComplicate this world you wrapped for ...,NaN,NO
97222,Almost Home,Mary Chapin Carpenter,97,NaN,NaN,7/12/99,NaN,"Country,Pop","Annie roboff, Beth nielsen chapman, Mary chapi...",Almost Home \nI saw my life this morning\nLyin...,NaN,NO
97223,Vivrant Thing,Q,98,NaN,NaN,7/12/99,NaN,Rap,"Q tip, J dilla, Barry white",Vivrant Thing \nUh check it out now\nUh no dou...,NaN,NO


In [5]:
result_df = billboard_df.drop_duplicates('name', keep='first')
result_df.to_csv("drop_duplicates.csv")

In [6]:
cleaned_billboard_df = result_df.drop(["weekly_rank", "writing_credits", "lyrics", "grammy_win"],axis=1)
cleaned_billboard_df

,name,artists,peak_position,weeks_on_chart,week,date,genre,features
0,Old Town Road,"Lil Nas,",1.0,7.0,7/6/19,5-Apr-19,"Country,Atlanta,Alternative Country,Hip-Hop,Tr...",Billy Ray Cyrus
1,Senorita,"Shawn Mendes, Camila Cabello",NaN,NaN,7/6/19,21-Jun-19,Pop,NaN
2,Bad Guy,Billie Eilish,2.0,13.0,7/6/19,29-Mar-19,"Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...",NaN
3,Talk,Khalid,3.0,20.0,7/6/19,7-Feb-19,"Synth-Pop,Pop",NaN
4,I Don't Care,"Ed Sheeran, Justin Bieber",2.0,7.0,7/6/19,10-May-19,"Canada,UK,Dance,Dance-Pop,Pop",NaN
...,...,...,...,...,...,...,...,...
97122,Praise You,Fatboy Slim,36.0,20.0,7/19/99,NaN,"Techno,Pop",NaN
97124,Anyone Else,Collin Raye,37.0,20.0,7/19/99,NaN,Country,NaN
97125,Nann,Trick Daddy,62.0,20.0,7/19/99,NaN,Rap,Trina
97211,She's Always Right,Clay Walker,74.0,6.0,7/12/99,NaN,Pop,NaN


In [7]:
cleaned_billboard_df.to_csv("cleaned_billboard_df.csv")

In [8]:
grammy_data = "grammy_songs.csv"
grammy_df = pd.read_csv(grammy_data)
grammy_df

,Unnamed: 0,X,GrammyAward,GrammyYear,Genre,Name,Artist
0,1,0,Record Of The Year,2018,General,this is America,Childish Gambino
1,2,1,Song Of The Year,2018,General,this is America,Childish Gambino
2,3,2,Best Pop Solo Performance,2018,Pop,Joanne (where Do you Think You're Goin'?),Lady Gaga
3,4,3,Best Pop Duo/Group Performance,2018,Pop,Shallow,Lady Gaga & Bradley Cooper
4,5,4,Best Dance Recording,2018,Dance/Electronic Music,Electricity,Silk City & Dua Lipa Featuring Diplo & Mark Ro...
...,...,...,...,...,...,...,...
423,424,423,Best Rap Solo Performance,1999,Rap,My Name is,Eminem
424,425,424,Best Rap Performance By A Duo Or Group,1999,Rap,you Got Me,The roots featuring Erykah Badu
425,426,425,Best Female Country Vocal Performance,1999,Country,Man! i Feel Like a Woman!,Shania Twain
426,427,426,Best Male Country Vocal Performance,1999,Country,Choices,George Jones


In [9]:
grammy_wins = grammy_df.groupby("Name").count()
grammy_wins 

,Unnamed: 0,X,GrammyAward,GrammyYear,Genre,Artist
Name,,,,,,
'Til Summer Comes Around,1,1,1,1,1,1
24 Frames,1,1,1,1,1,1
24k Magic,1,1,1,1,1,1
500 Miles High,1,1,1,1,1,1
99 Problems,1,1,1,1,1,1
...,...,...,...,...,...,...
you Don't Know My Name,1,1,1,1,1,1
you Got Me,1,1,1,1,1,1
you Say,1,1,1,1,1,1


In [10]:
grammy_wins = grammy_wins.drop(["Unnamed: 0", "X", "GrammyYear", "Genre", "Artist"],axis=1)
grammy_wins

,GrammyAward
Name,
'Til Summer Comes Around,1
24 Frames,1
24k Magic,1
500 Miles High,1
99 Problems,1
...,...
you Don't Know My Name,1
you Got Me,1
you Say,1


In [11]:
grammy_wins.to_csv("grammy_wins.csv")

In [12]:
grammy_genre = grammy_df.drop(["Unnamed: 0", "X", "GrammyAward", "Artist"],axis=1)
grammy_genre

,GrammyYear,Genre,Name
0,2018,General,this is America
1,2018,General,this is America
2,2018,Pop,Joanne (where Do you Think You're Goin'?)
3,2018,Pop,Shallow
4,2018,Dance/Electronic Music,Electricity
...,...,...,...
423,1999,Rap,My Name is
424,1999,Rap,you Got Me
425,1999,Country,Man! i Feel Like a Woman!
426,1999,Country,Choices


In [13]:
grammy_genre.to_csv("grammy_genre.csv")